## Visual Inspection of intersections including waterways shapefiles

In [51]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from haversine import haversine, Unit

In [52]:
water = gpd.read_file('../data/raw/waterways.shp')
water.to_crs(epsg=4326, inplace=True)

In [53]:
documented_points = pd.read_csv('../data/semi-processed/intersections_to_check_final.csv')
documented_points['road_to'].value_counts()

road_to
Z1031    2
N207     2
R240     2
N102     2
Z1034    2
Z1005    1
R241     1
N204     1
R220     1
R360     1
R310     1
R301     1
R203     1
R170     1
N104     1
Z1065    1
R151     1
Z1048    1
Z1042    1
Z1044    1
Z1402    1
N2       1
N105     1
Z2013    1
Name: count, dtype: int64

In [54]:
probable_points = pd.read_csv('../data/semi-processed/probable_intersections_to_check_final.csv')
probable_points

,Unnamed: 0,road,LRPName,lat,lon,chainage,road_to,closest_lrp,closest_lat,closest_lon,distance_to_closest
0,0,N1,LRPS,23.706028,90.443333,0.000,N8,LRPS,23.702139,90.435361,919.684490
1,2429,N1,LRP094a,23.419138,91.172194,91.583,R140,LRP004a,23.419166,91.172333,14.505865
2,7224,N1,LRP231a,22.399611,91.757333,228.406,Z1084,LRPE,22.397583,91.750917,697.136864
3,14767,N2,LRP037,23.922666,90.703639,36.266,R114,LRP028,23.919749,90.704778,344.363315
4,10966,N2,LRP229a,24.878666,91.877138,227.988,R250,LRP002a,24.878860,91.877249,24.348922


In [55]:
roads = pd.read_csv('../data/raw/_roads3.csv')
roads.rename(columns={'lrp':'LRPName'}, inplace=True)
roads

,road,chainage,LRPName,lat,lon,gap,type,name
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...
1,N1,0.814,LRPSa,23.702917,90.450417,NaN,Culvert,Box Culvert
2,N1,0.822,LRPSb,23.702778,90.450472,NaN,CrossRoad,Intersection with Z1101
3,N1,1.000,LRP001,23.702139,90.451972,NaN,KmPost,Km post missing
4,N1,2.000,LRP002,23.697889,90.460583,NaN,KmPost,Km post missing
...,...,...,...,...,...,...,...,...
51343,Z8943,7.000,LRP007,22.429499,90.785722,NaN,KmPost,Km Post Missing
51344,Z8943,8.000,LRP008,22.430110,90.776333,NaN,KmPost,Km Post Missing
51345,Z8943,8.133,LRP008a,22.430249,90.774888,BS,Bridge,Start of Bridge
51346,Z8943,8.151,LRP008b,22.430249,90.774860,BE,Bridge,End of Bridge


In [56]:
# Make a function that considers which rows to plot out of a dataframe
def plot_intersection_data(df, start_idx, end_idx, buffer_size):
    for idx, row in df[start_idx:end_idx].iterrows():
        # Create a bounding box geometry for each pair of coordinates
        min_lon = min(row['lon'], row['closest_lon']) - buffer_size
        min_lat = min(row['lat'], row['closest_lat']) - buffer_size
        max_lon = max(row['lon'], row['closest_lon']) + buffer_size
        max_lat = max(row['lat'], row['closest_lat']) + buffer_size
        
        # Only waterways withing the bounds
        water_in_box = water.cx[min_lon:max_lon, min_lat:max_lat]
        
        # Plot waterways for the bound-box thingy
        water_in_box.plot(color='blue', linewidth=0.9)  # Plot waterways
        
        # Make clear what each pair is and what the distance between the pointsis
        print('Intersection', idx,' between ', row['road'],' and ',row['road_to'] )
        print('Distance inbetween points: ',row['distance_to_closest'])
        print('Coordinates point 1:', row['closest_lat'],row['closest_lon'])
        print('Coordinates point 2:', row['lat'],row['lon'])
        
        # Plot both roads you wanted each other color, not blue
        road = row['road']
        road_to = row['road_to']
        plt.plot(roads[roads['road']==road]['lon'], roads[roads['road']==road]['lat'], color='green')  # Plot road
        plt.plot(roads[roads['road']==road_to]['lon'], roads[roads['road']==road_to]['lat'], color='red')  # Plot road_to
        
        # Plot the coordinate points in the road colors
        plt.plot(row['lon'], row['lat'], 'go', label='lat, lon point')  # Plot lat, lon point
        plt.plot(row['closest_lon'], row['closest_lat'], 'ro', label='closest_lat, closest_lon point')  # Plot closest_lat, closest_lon point
        
        # Show within the boundaries aka bounding box
        bounding_box = box(min_lon, min_lat, max_lon, max_lat)
        x, y = bounding_box.exterior.xy
        plt.plot(x, y, color='blue', alpha=0.7)
        
        # Set plot limits according to the bounding box
        plt.xlim(min_lon, max_lon)
        plt.ylim(min_lat, max_lat)
        
        plt.show()

In [ ]:
plot_intersection_data( documented_points,0, 22, 0.05)

In [ ]:
plot_intersection_data(documented_points,22, 23, 0.5)

In [ ]:
plot_intersection_data(documented_points, 23, 29, 0.05)

## Conclusion:
Of all considered intersection points in the assignment, only the intersection between N1 and Z1065 will not get considered.

In [57]:
documented_points_final = documented_points.drop(13, inplace=False)

In [58]:
documented_points_final['road_to'].value_counts()

road_to
Z1031    2
N207     2
R240     2
N102     2
Z1034    2
R203     1
R241     1
N204     1
R220     1
R360     1
R310     1
R301     1
R170     1
Z1005    1
N104     1
R151     1
Z1048    1
Z1042    1
Z1044    1
Z1402    1
N2       1
N105     1
Z2013    1
Name: count, dtype: int64

## Make plots for all the probable intersections

In [ ]:
plot_intersection_data(probable_points, 0, 5, 0.05)

## Inspect ind 1 and 4 closer:

In [ ]:
plot_intersection_data(probable_points, 1, 2, 0.04)

In [ ]:
plot_intersection_data(probable_points, 4, 5, 0.005)

## Conclusion
From the visually inspected probable intersections, only 2 are going to be considered in the dataset: the intersection between N1-R140 and N2-R250 due to their short distances and the absence of water bodies inbetween them according to the shapefiles.

Although there were also no waterbodies inbetween the remaining three intersections, their long distance (hundreds of kilometers) resulted into them being removed from the dataset.

In [59]:
probable_points

,Unnamed: 0,road,LRPName,lat,lon,chainage,road_to,closest_lrp,closest_lat,closest_lon,distance_to_closest
0,0,N1,LRPS,23.706028,90.443333,0.000,N8,LRPS,23.702139,90.435361,919.684490
1,2429,N1,LRP094a,23.419138,91.172194,91.583,R140,LRP004a,23.419166,91.172333,14.505865
2,7224,N1,LRP231a,22.399611,91.757333,228.406,Z1084,LRPE,22.397583,91.750917,697.136864
3,14767,N2,LRP037,23.922666,90.703639,36.266,R114,LRP028,23.919749,90.704778,344.363315
4,10966,N2,LRP229a,24.878666,91.877138,227.988,R250,LRP002a,24.878860,91.877249,24.348922


In [60]:
probable_points_final = probable_points.drop([0,2,3], inplace=False)
probable_points_final

,Unnamed: 0,road,LRPName,lat,lon,chainage,road_to,closest_lrp,closest_lat,closest_lon,distance_to_closest
1,2429,N1,LRP094a,23.419138,91.172194,91.583,R140,LRP004a,23.419166,91.172333,14.505865
4,10966,N2,LRP229a,24.878666,91.877138,227.988,R250,LRP002a,24.878860,91.877249,24.348922


## Make plots for the extra intersection points

In [61]:
extra_intersections = pd.read_csv('../data/semi-processed/probable_crossings_extra.csv')
extra_intersections

,Unnamed: 0,road,LRPName,lat,lon,chainage,road_to,closest_lrp,closest_lat,closest_lon,distance_to_closest
0,2428,N1,LRP094,23.419527,91.171416,91.493,R140,LRP004a,23.419166,91.172333,101.789456
1,10961,N2,LRP228,24.877110,91.872944,227.493,R250,LRP002,24.879388,91.876860,469.307508
2,10962,N2,LRP228a,24.877222,91.873527,227.568,R250,LRP002,24.879388,91.876860,413.648937
3,10963,N2,LRP228b,24.877222,91.873722,227.587,R250,LRP002,24.879388,91.876860,397.863741
4,10964,N2,LRP228c,24.877694,91.875277,227.756,R250,LRP002a,24.878860,91.877249,237.503563
5,10965,N2,LRP229,24.877849,91.875574,227.793,R250,LRP002a,24.878860,91.877249,203.014909
6,10967,N2,LRP229b,24.880194,91.878249,228.186,R250,LRP002,24.879388,91.876860,166.294850
7,10968,N2,LRP229c,24.882749,91.879694,228.504,R250,LRP001,24.881833,91.875444,440.660341


In [ ]:
plot_intersection_data(extra_intersections,0, 8, 0.04)

Keep indexes 0 

In [62]:
extra_points_final = extra_intersections.loc[[0]]
extra_points_final

,Unnamed: 0,road,LRPName,lat,lon,chainage,road_to,closest_lrp,closest_lat,closest_lon,distance_to_closest
0,2428,N1,LRP094,23.419527,91.171416,91.493,R140,LRP004a,23.419166,91.172333,101.789456


# Concat the final intersection points

In [63]:
inspected_intersections = pd.concat([documented_points_final, probable_points_final, extra_points_final], ignore_index=True)

In [64]:
inspected_intersections.drop(inspected_intersections.columns[:1], axis=1, inplace=True)
inspected_intersections

,road,chainage,LRPName,lat,lon,gap,type,name,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,model_type
0,N1,140.251,LRP143a,23.047944,91.365139,NaN,crossroad,Z1031,Z1031,4.200797,23.047972,91.365167,LRP001b,End of overlop with N1,Others,1.496,intersection
1,N1,145.409,LRP148a,23.009556,91.381360,NaN,crossroad / kmpost,Intersection with N104 / Km post missing,N104,9.078352,23.009528,91.381444,LRP001a,Right to Comilla Left to Chittagong,CrossRoad,1.878,intersection
2,N2,11.464,LRP012a,23.785333,90.568555,NaN,crossroad,Intersection with N105,N105,29.759391,23.785194,90.568805,LRP012a,N2 Dhaka-Sylhet -Jaflong Road,CrossRoad,12.803,intersection
3,N1,8.763,LRP009a,23.706083,90.521527,NaN,"sideroad,left",Road to Sylhet (N2),N2,20.374160,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.000,intersection
4,N1,45.322,LRP046a,23.530749,90.781610,NaN,"sideroad,right","Z1402, road to Pennai",Z1402,0.000000,23.530749,90.781610,LRPE,Meet with N1 at Pennai,Others,43.849,intersection
5,N1,47.224,LRP047a,23.530694,90.792444,NaN,"sideroad,right",Z1044,Z1044,0.000000,23.530694,90.792444,LRPS,Start of Road From N1 at Gouripur,Others,0.000,intersection
6,N1,55.477,LRP056c,23.512667,90.871416,NaN,"sideroad,left","Z1042, road to Bancharampur",Z1042,3.091223,23.512639,90.871416,LRPS,Start of Road from N1 at Eliotganj.,Others,0.000,intersection
7,N1,81.846,LRP084a,23.478972,91.118166,NaN,"sideroad,left",Road to Brahmanbaria(N102),N102,2.835290,23.478972,91.118194,LRPS,Start of road from N1 Mainamati,Others,0.000,intersection
8,N1,128.009,LRP130a,23.149694,91.325499,NaN,"sideroad,right",Road to Nangolhat (Z1048),Z1048,31.387443,23.149722,91.325194,LRPE,End of road at Chewara Intersection with N1.,Others,31.130,intersection
9,N1,138.869,LRP142a,23.061472,91.361667,NaN,"sideroad,right",Road to Karerhat (Z1031),Z1031,0.000000,23.061472,91.361667,LRPS,Start from Rahman filling station at Mohhamad ...,Others,0.000,intersection


In [65]:
inspected_intersections_df = inspected_intersections.copy()

new_rows = []

# Iterate over each row in the intersections that are documented and above the threshold
for index, row in inspected_intersections_df.iterrows():
    # Extract coordinates of the intersection [on the N1 or N2]
    intersection_coords = (row['lat'], row['lon'])
    
    # Filter roads df to get points on road_to
    road_to_points = roads[roads['road'] == row['road_to']]
    
    # Calculate distances between the intersection and all road_to points using haversine 
    distances = [haversine(intersection_coords, (lat, lon), unit=Unit.METERS) for lat, lon in zip(road_to_points['lat'], road_to_points['lon'])]
    
    # The minimum distance index is needed
    closest_point_index = distances.index(min(distances))
    
    # Get the closest point's columnss [of the 'smaller' road]
    closest_lrp_name = road_to_points.iloc[closest_point_index]['LRPName']
    closest_type= road_to_points.iloc[closest_point_index]['type']
    closest_name = road_to_points.iloc[closest_point_index]['name']
    closest_lat = road_to_points.iloc[closest_point_index]['lat']
    closest_lon = road_to_points.iloc[closest_point_index]['lon']
    closest_chainage = road_to_points.iloc[closest_point_index]['chainage']
    
    # Calculate distance between the intersection and the closest point
    distance_to_closest = distances[closest_point_index]
    
    # Create a new row with information about the closest point
    new_row = {
        'road': row['road_to'],
        'closest_chainage':closest_chainage,
        'lat' :row['lat'],
        'lon' :row['lon'],
        'chainage':row['chainage'],
        'LRPName': closest_lrp_name,
        'road_to':row['road'],
        'closest_lrp':row['LRPName'],
        'closest_lat': closest_lat,
        'closest_lon': closest_lon,
        'distance_to_closest': distance_to_closest,
        'type':closest_type,
        'name':closest_name
    }
    
    new_rows.append(new_row)

intersection_point_small_road = pd.DataFrame(new_rows)

# Concatenate intersections as mentioned  on N1orN2  with intersections as mentioned in the smaller roads
documented_intersections_final = pd.concat([inspected_intersections_df, intersection_point_small_road], ignore_index=True)

documented_intersections_final['model_type']= 'intersection'
documented_intersections_final

,road,chainage,LRPName,lat,lon,gap,type,name,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,model_type
0,N1,140.251,LRP143a,23.047944,91.365139,NaN,crossroad,Z1031,Z1031,4.200797,23.047972,91.365167,LRP001b,End of overlop with N1,Others,1.496,intersection
1,N1,145.409,LRP148a,23.009556,91.381360,NaN,crossroad / kmpost,Intersection with N104 / Km post missing,N104,9.078352,23.009528,91.381444,LRP001a,Right to Comilla Left to Chittagong,CrossRoad,1.878,intersection
2,N2,11.464,LRP012a,23.785333,90.568555,NaN,crossroad,Intersection with N105,N105,29.759391,23.785194,90.568805,LRP012a,N2 Dhaka-Sylhet -Jaflong Road,CrossRoad,12.803,intersection
3,N1,8.763,LRP009a,23.706083,90.521527,NaN,"sideroad,left",Road to Sylhet (N2),N2,20.374160,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.000,intersection
4,N1,45.322,LRP046a,23.530749,90.781610,NaN,"sideroad,right","Z1402, road to Pennai",Z1402,0.000000,23.530749,90.781610,LRPE,Meet with N1 at Pennai,Others,43.849,intersection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,N207,190.892,LRPE,24.626499,91.677499,NaN,Others,Road end with N2 at Mukti joddha Chattor Sherpur.,N2,18.737016,24.626333,91.677527,LRP191b,NaN,NaN,67.985,intersection
58,Z2013,225.373,LRPS,24.867305,91.856806,NaN,Others,Road Start From N2 at Chandipool Sylhet,N2,0.000000,24.867305,91.856806,LRP225b,NaN,NaN,0.000,intersection
59,R140,91.583,LRP004a,23.419138,91.172194,NaN,CrossRoad,Intersection N1 at Paduar Bazar,N1,14.505865,23.419166,91.172333,LRP094a,NaN,NaN,4.745,intersection
60,R250,227.988,LRP002a,24.878666,91.877138,NaN,CrossRoad,N2 (Over pass),N2,24.348922,24.878860,91.877249,LRP229a,NaN,NaN,1.389,intersection


In [67]:
documented_intersections_final[['road','LRPName','road_to','closest_lrp','lat','closest_lat','lon','closest_lon']].head(20)

,road,LRPName,road_to,closest_lrp,lat,closest_lat,lon,closest_lon
0,N1,LRP143a,Z1031,LRP001b,23.047944,23.047972,91.365139,91.365167
1,N1,LRP148a,N104,LRP001a,23.009556,23.009528,91.381360,91.381444
2,N2,LRP012a,N105,LRP012a,23.785333,23.785194,90.568555,90.568805
3,N1,LRP009a,N2,LRPS,23.706083,23.705917,90.521527,90.521444
4,N1,LRP046a,Z1402,LRPE,23.530749,23.530749,90.781610,90.781610
5,N1,LRP047a,Z1044,LRPS,23.530694,23.530694,90.792444,90.792444
6,N1,LRP056c,Z1042,LRPS,23.512667,23.512639,90.871416,90.871416
7,N1,LRP084a,N102,LRPS,23.478972,23.478972,91.118166,91.118194
8,N1,LRP130a,Z1048,LRPE,23.149694,23.149722,91.325499,91.325194
9,N1,LRP142a,Z1031,LRPS,23.061472,23.061472,91.361667,91.361667


In [68]:
roads_in_simulation = documented_intersections_final.road.unique().tolist()
roads_in_simulation

['N1',
 'N2',
 'Z1031',
 'N104',
 'N105',
 'Z1402',
 'Z1044',
 'Z1042',
 'N102',
 'Z1048',
 'Z1034',
 'R151',
 'R170',
 'Z1005',
 'R203',
 'R301',
 'R310',
 'R360',
 'R220',
 'N204',
 'R240',
 'N207',
 'R241',
 'Z2013',
 'R140',
 'R250']

## Now lets merge the intersection points with the start/end points

In [69]:
start_end = pd.DataFrame()

for r in roads['road'].unique():
    road_of_interest = roads[roads['road']==r]
    first_row = road_of_interest.head(1)
    last_row = road_of_interest.tail(1)
    start_end = pd.concat([start_end, first_row, last_row])
    
start_end = start_end.reset_index(drop=True)
start_end['model_type']= 'sourcesink'
start_end

,road,chainage,LRPName,lat,lon,gap,type,name,model_type
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,sourcesink
1,N1,462.254,LRPE,20.862917,92.298083,NaN,Others,"End of Road at Shapla Chattar ,Teknaf Meet wit...",sourcesink
2,N101,0.000,LRPS,23.454139,91.212861,NaN,Others,Start of Road from N120 at Balutopa,sourcesink
3,N101,6.021,LRPE,23.459306,91.253389,NaN,Others,End of road Bibir bazar Bridge,sourcesink
4,N102,0.000,LRPS,23.478972,91.118194,NaN,Others,Start of road from N1 Mainamati,sourcesink
...,...,...,...,...,...,...,...,...,...
1689,Z8915,15.660,LRPE,22.599694,90.753167,NaN,Others,"End of road at Daulatkhan, Bhola.",sourcesink
1690,Z8916,0.000,LRPS,22.625499,90.661722,NaN,Others,Start from R890 at Guingerhat Bazar Dhola,sourcesink
1691,Z8916,11.591,LRPE,22.597889,90.746666,NaN,Others,End at Selim Chattar at Intersect with Z8915.,sourcesink
1692,Z8943,0.000,LRPS,22.426444,90.849472,NaN,Others,Start at intersection of Z8905 /BC joint infro...,sourcesink


In [71]:
start_end_intersections2 = start_end[start_end['road'].isin(roads_in_simulation)]
start_end_intersections2

,road,chainage,LRPName,lat,lon,gap,type,name,model_type
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,sourcesink
1,N1,462.254,LRPE,20.862917,92.298083,NaN,Others,"End of Road at Shapla Chattar ,Teknaf Meet wit...",sourcesink
4,N102,0.000,LRPS,23.478972,91.118194,NaN,Others,Start of road from N1 Mainamati,sourcesink
5,N102,82.682,LRPE,24.050611,91.114667,NaN,Others,Meet with N 2 at Sanail,sourcesink
8,N104,0.000,LRPS,23.009667,91.399416,NaN,Others,Intersection with Z1031,sourcesink
9,N104,49.630,LRPE,22.825749,91.101444,NaN,Others,Meet with Z1441& Z1405 at Sonapur.,sourcesink
10,N105,0.000,LRPS,23.690416,90.546611,NaN,Others,"Starts of road from N1,Dhaka teknaf road at Ma...",sourcesink
11,N105,48.870,LRP048,23.989527,90.358222,NaN,KmPost / Others,"B.Bridge 83 km,Tangail 60 km / End of Road In...",sourcesink
36,N2,0.000,LRPS,23.705917,90.521444,NaN,Others,Road Start from N1 at Katchpur,sourcesink
37,N2,286.516,LRPE,25.157056,92.017638,NaN,Others,Road End at Jaflong.,sourcesink


In [72]:
start_end_intersections = start_end_intersections2.copy()

In [82]:
documented_intersections_final

,road,chainage,LRPName,lat,lon,gap,type,name,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,model_type
0,N1,140.251,LRP143a,23.047944,91.365139,NaN,crossroad,Z1031,Z1031,4.200797,23.047972,91.365167,LRP001b,End of overlop with N1,Others,1.496,intersection
1,N1,145.409,LRP148a,23.009556,91.381360,NaN,crossroad / kmpost,Intersection with N104 / Km post missing,N104,9.078352,23.009528,91.381444,LRP001a,Right to Comilla Left to Chittagong,CrossRoad,1.878,intersection
2,N2,11.464,LRP012a,23.785333,90.568555,NaN,crossroad,Intersection with N105,N105,29.759391,23.785194,90.568805,LRP012a,N2 Dhaka-Sylhet -Jaflong Road,CrossRoad,12.803,intersection
3,N1,8.763,LRP009a,23.706083,90.521527,NaN,"sideroad,left",Road to Sylhet (N2),N2,20.374160,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.000,intersection
4,N1,45.322,LRP046a,23.530749,90.781610,NaN,"sideroad,right","Z1402, road to Pennai",Z1402,0.000000,23.530749,90.781610,LRPE,Meet with N1 at Pennai,Others,43.849,intersection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,N207,190.892,LRPE,24.626499,91.677499,NaN,Others,Road end with N2 at Mukti joddha Chattor Sherpur.,N2,18.737016,24.626333,91.677527,LRP191b,NaN,NaN,67.985,intersection
58,Z2013,225.373,LRPS,24.867305,91.856806,NaN,Others,Road Start From N2 at Chandipool Sylhet,N2,0.000000,24.867305,91.856806,LRP225b,NaN,NaN,0.000,intersection
59,R140,91.583,LRP004a,23.419138,91.172194,NaN,CrossRoad,Intersection N1 at Paduar Bazar,N1,14.505865,23.419166,91.172333,LRP094a,NaN,NaN,4.745,intersection
60,R250,227.988,LRP002a,24.878666,91.877138,NaN,CrossRoad,N2 (Over pass),N2,24.348922,24.878860,91.877249,LRP229a,NaN,NaN,1.389,intersection


## Hoi Em, ik snapte niet helemaal waarom de lat en lons werden aangepast dus heb het nu zo gedaan. Met de df hieronder is eigenlijk het enige wat er moet gebeuren dat die df gemerged wordt met de originele (weet niet goed welke dat is met alle notebooks) en dat de rijen overruled worden. Desnoods als het niet lukt doen we het met de hand, dan kan ik morgen de analyse doen.

In [90]:
## Correct the coordinates of the LRPS and LRPEs in case they are also an intersection that was moved
for i in start_end_intersections.index:
    for x in documented_intersections_final.index:
        if (start_end_intersections.at[i, 'road']== documented_intersections_final.at[x,'road']) and (start_end_intersections.at[i, 'LRPName']== documented_intersections_final.at[x,'LRPName']):
            print(start_end_intersections.at[i, 'road'], start_end_intersections.at[i, 'LRPName'])
            start_end_intersections.at[i,'model_type']= 'intersourcesink'

N102 LRPS
N102 LRPE
N2 LRPS
N204 LRPS
N207 LRPS
N207 LRPE
R151 LRPS
R170 LRPS
R203 LRPS
R220 LRPS
R240 LRPS
R241 LRPS
R301 LRPE
R310 LRPE
R360 LRPE
Z1005 LRPS
Z1031 LRPS
Z1042 LRPS
Z1044 LRPS
Z1048 LRPE
Z1402 LRPE
Z2013 LRPS


In [91]:
start_end_intersections # DEZE KLOPT NU

,road,chainage,LRPName,lat,lon,gap,type,name,model_type
0,N1,0.000,LRPS,23.706028,90.443333,NaN,Others,Start of Road after Jatrabari Flyover infront...,sourcesink
1,N1,462.254,LRPE,20.862917,92.298083,NaN,Others,"End of Road at Shapla Chattar ,Teknaf Meet wit...",sourcesink
4,N102,0.000,LRPS,23.478972,91.118166,NaN,Others,Start of road from N1 Mainamati,intersourcesink
5,N102,82.682,LRPE,24.050833,91.114444,NaN,Others,Meet with N 2 at Sanail,intersourcesink
8,N104,0.000,LRPS,23.009667,91.399416,NaN,Others,Intersection with Z1031,sourcesink
9,N104,49.630,LRPE,22.825749,91.101444,NaN,Others,Meet with Z1441& Z1405 at Sonapur.,sourcesink
10,N105,0.000,LRPS,23.690416,90.546611,NaN,Others,"Starts of road from N1,Dhaka teknaf road at Ma...",sourcesink
11,N105,48.870,LRP048,23.989527,90.358222,NaN,KmPost / Others,"B.Bridge 83 km,Tangail 60 km / End of Road In...",sourcesink
36,N2,0.000,LRPS,23.706083,90.521527,NaN,Others,Road Start from N1 at Katchpur,intersourcesink
37,N2,286.516,LRPE,25.157056,92.017638,NaN,Others,Road End at Jaflong.,sourcesink


## Omdat de chainages hier niet kloppen kom je later in de knel, ik zit niet genoeg in de notebooks om te snappen waarom het zo gaat. Snap ook niet helemaal waarom het zo complex gaat ipv de manier hierboven :/

In [86]:
documented_intersections_final.head(60) # DEZE KLOPT NIET, heeft intersections dubbel. bv N2-Z2013 heeft chainage 225.373 maar intersection Z2013-N2 ook.

,road,chainage,LRPName,lat,lon,gap,type,name,road_to,distance_to_closest,closest_lat,closest_lon,closest_lrp,closest_name,closest_type,closest_chainage,model_type
0,N1,140.251,LRP143a,23.047944,91.365139,NaN,crossroad,Z1031,Z1031,4.200797,23.047972,91.365167,LRP001b,End of overlop with N1,Others,1.496,intersection
1,N1,145.409,LRP148a,23.009556,91.381360,NaN,crossroad / kmpost,Intersection with N104 / Km post missing,N104,9.078352,23.009528,91.381444,LRP001a,Right to Comilla Left to Chittagong,CrossRoad,1.878,intersection
2,N2,11.464,LRP012a,23.785333,90.568555,NaN,crossroad,Intersection with N105,N105,29.759391,23.785194,90.568805,LRP012a,N2 Dhaka-Sylhet -Jaflong Road,CrossRoad,12.803,intersection
3,N1,8.763,LRP009a,23.706083,90.521527,NaN,"sideroad,left",Road to Sylhet (N2),N2,20.374160,23.705917,90.521444,LRPS,Road Start from N1 at Katchpur,Others,0.000,intersection
4,N1,45.322,LRP046a,23.530749,90.781610,NaN,"sideroad,right","Z1402, road to Pennai",Z1402,0.000000,23.530749,90.781610,LRPE,Meet with N1 at Pennai,Others,43.849,intersection
5,N1,47.224,LRP047a,23.530694,90.792444,NaN,"sideroad,right",Z1044,Z1044,0.000000,23.530694,90.792444,LRPS,Start of Road From N1 at Gouripur,Others,0.000,intersection
6,N1,55.477,LRP056c,23.512667,90.871416,NaN,"sideroad,left","Z1042, road to Bancharampur",Z1042,3.091223,23.512639,90.871416,LRPS,Start of Road from N1 at Eliotganj.,Others,0.000,intersection
7,N1,81.846,LRP084a,23.478972,91.118166,NaN,"sideroad,left",Road to Brahmanbaria(N102),N102,2.835290,23.478972,91.118194,LRPS,Start of road from N1 Mainamati,Others,0.000,intersection
8,N1,128.009,LRP130a,23.149694,91.325499,NaN,"sideroad,right",Road to Nangolhat (Z1048),Z1048,31.387443,23.149722,91.325194,LRPE,End of road at Chewara Intersection with N1.,Others,31.130,intersection
9,N1,138.869,LRP142a,23.061472,91.361667,NaN,"sideroad,right",Road to Karerhat (Z1031),Z1031,0.000000,23.061472,91.361667,LRPS,Start from Rahman filling station at Mohhamad ...,Others,0.000,intersection


## Eigenlijk wil je nu alles wat een LRPS is en wat niet een chainage van 0 heeft weg hebben

In [89]:
documented_intersections_start_end = pd.concat([start_end_intersections, documented_intersections_final], ignore_index=True)
documented_intersections_start_end.sort_values(by=['road','chainage', 'model_type'], inplace=True)
documented_intersections_start_end[['road','LRPName','chainage','model_type','lat','lon']].head(30)

,road,LRPName,chainage,model_type,lat,lon
0,N1,LRPS,0.000,sourcesink,23.706028,90.443333
55,N1,LRP009a,8.763,intersection,23.706083,90.521527
56,N1,LRP046a,45.322,intersection,23.530749,90.781610
57,N1,LRP047a,47.224,intersection,23.530694,90.792444
58,N1,LRP056c,55.477,intersection,23.512667,90.871416
59,N1,LRP084a,81.846,intersection,23.478972,91.118166
82,N1,LRP094,91.493,intersection,23.419527,91.171416
80,N1,LRP094a,91.583,intersection,23.419138,91.172194
60,N1,LRP130a,128.009,intersection,23.149694,91.325499
61,N1,LRP142a,138.869,intersection,23.061472,91.361667


In [ ]:
# Value counts for lat en lon
lat_counts = documented_intersections_start_end['lat'].value_counts()
lon_counts = documented_intersections_start_end['lon'].value_counts()

#Filter rows where 'lat' AND 'lon' occur three times
filtered_df = documented_intersections_start_end[
    (documented_intersections_start_end['lat'].isin(lat_counts.index[lat_counts == 3]) | documented_intersections_start_end['lon'].isin(lon_counts.index[lon_counts == 3]))  
]

filtered_df

In [ ]:
filtered_df.sort_values(by='lat', ascending=False, inplace=True)
filtered_df['trios']='trio'
filtered_df.reset_index(inplace=True)#[['road','road_to','model_type','trios']]
filtered_df

In [ ]:
# Calculate the group labels
group_labels = [(i // 3) + 1 for i in range(len(filtered_df))]

# Assign the group labels to a new column
filtered_df['trios'] = ['group ' + str(label) for label in group_labels]
    
filtered_df[['road','road_to','model_type','trios','lat','lon']]
filtered_df

In [ ]:
for i in filtered_df.index:
    if filtered_df.at[i,'model_type']=='sourcesink':
        filtered_df.at[i,'model_type']='intersourcesink'
        
filtered_df[['index','road','road_to','model_type','trios','lat','lon']].head(20)

In [ ]:
documented_intersections_start_end[['road', 'road_to', 'model_type']]

In [ ]:
documented_intersections_start_end_final = documented_intersections_start_end.copy()
documented_intersections_start_end_final['trios'] = filtered_df['trios']
documented_intersections_start_end_final['model_type_correct'] = '0'

filtered_index_values = filtered_df.index.values

for i, row in documented_intersections_start_end_final.iterrows():
    if i in filtered_index_values:
        if filtered_df.at[i, 'model_type'] == 'intersourcesink':
            documented_intersections_start_end_final.at[i, 'model_type_correct'] = 'intersourcesink'
        else:
            documented_intersections_start_end_final.at[i, 'model_type_correct'] = 'sourcesink'
    else:
        documented_intersections_start_end_final.at[i, 'model_type_correct'] = documented_intersections_start_end_final.at[i, 'model_type']
    print(i,(i in filtered_index_values), documented_intersections_start_end_final.at[i, 'model_type_correct'] )

In [ ]:
documented_intersections_start_end_final.model_type_correct.value_counts()

In [ ]:
documented_intersections_start_end_final[['trios']]

In [ ]:
documented_intersections_start_end_final.model_type_correct.value_counts()

In [ ]:
# Model_type fix the column name
documented_intersections_start_end_final.drop(columns=['model_type'], inplace=True)
documented_intersections_start_end_final.rename(columns={'model_type_correct': 'model_type'}, inplace=True)

In [ ]:
# Extract to CSV to use in the 'Handle duplicate bridges' notebook
documented_intersections_start_end_final.to_csv('../data/semi-processed/identified_roads_final.csv')